In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.manifold import TSNE
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.metrics import r2_score
import random
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/myanimelist-dataset/user_anime000000000032.csv
/kaggle/input/myanimelist-dataset/user_anime000000000000.csv
/kaggle/input/myanimelist-dataset/user_anime000000000029.csv
/kaggle/input/myanimelist-dataset/user_anime000000000019.csv
/kaggle/input/myanimelist-dataset/user_anime000000000011.csv
/kaggle/input/myanimelist-dataset/user_anime000000000025.csv
/kaggle/input/myanimelist-dataset/user_anime000000000040.csv
/kaggle/input/myanimelist-dataset/user_anime000000000047.csv
/kaggle/input/myanimelist-dataset/user.csv
/kaggle/input/myanimelist-dataset/user_anime000000000002.csv
/kaggle/input/myanimelist-dataset/user_anime000000000009.csv
/kaggle/input/myanimelist-dataset/user_anime000000000033.csv
/kaggle/input/myanimelist-dataset/user_anime000000000006.csv
/kaggle/input/myanimelist-dataset/user_anime000000000068.csv
/kaggle/input/myanimelist-dataset/user_anime000000000034.csv
/kaggle/input/myanimelist-dataset/user_anime000000000061.csv
/kaggle/input/myanimelist-dataset/user_ani

****Collaborative filtering****

considerations -  
- extremely high number of users

In [18]:
anime  = pd.read_csv('../input/myanimelist-dataset/anime.csv', sep='\t')
anime_anime = pd.read_csv('../input/myanimelist-dataset/anime_anime.csv', sep='\t')
user = pd.read_csv('../input/myanimelist-dataset/user.csv', sep='\t')
user_user = pd.read_csv('../input/myanimelist-dataset/user_user.csv', sep='\t')

In [25]:
len(user)

1123284

In [ ]:
#anime loader
def user_anime_data(file_list,shuffle=True):
    
    
    
    

In [26]:
tf.ones((13379,1123284),tf.dtypes.int8)

2022-10-29 22:00:38.639570: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<tf.Tensor: shape=(13379, 1123284), dtype=int8, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int8)>

In [22]:
import os
os.listdir('../input/myanimelist-dataset/')
user_1 = pd.read_csv('../input/myanimelist-dataset/user_anime000000000032.csv', sep='\t')
user_1

,user_id,anime_id,favorite,review_id,review_date,review_num_useful,review_score,review_story_score,review_animation_score,review_sound_score,review_character_score,review_enjoyment_score,score,status,progress,last_interaction_date
0,icarocrdk,481,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,completed,224.0,2004-09-29 00:00:00
1,icarocrdk,482,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,completed,180.0,2008-03-26 00:00:00
2,icarocrdk,49926,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,completed,7.0,2021-11-28 00:00:00
3,icarocrdk,550,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,completed,27.0,2004-05-11 00:00:00
4,icarocrdk,552,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,watching,32.0,2022-02-23 12:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3600667,irohaasu,31553,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,completed,1.0,2016-03-30 00:00:00
3600668,irohaasu,32281,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,completed,1.0,2016-08-26 00:00:00
3600669,irohaasu,32901,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,plan_to_watch,0.0,2017-06-25 00:00:00
3600670,irohaasu,33487,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,completed,12.0,2017-03-23 00:00:00
